In [ ]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `qwiklabs-gcp-02-3205ce60aeba.emergency_response`;

DROP TABLE IF EXISTS `qwiklabs-gcp-02-3205ce60aeba.emergency_response.emergency_calls_response_times`;
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-3205ce60aeba.emergency_response.emergency_calls_response_times`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv'],
  skip_leading_rows = 1
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `qwiklabs-gcp-02-3205ce60aeba.emergency_response.response_time_model`
OPTIONS(
  model_type='linear_reg',
  input_label_cols=['response_time']
) AS
SELECT
  call_id,
  call_type,
  location,
  weather_condition,
  day_of_week,
  time_of_day,
  traffic_level,
  distance_to_station,
  units_available,
  response_time
FROM
  `qwiklabs-gcp-02-3205ce60aeba.emergency_response.emergency_calls_response_times`
WHERE
  response_time IS NOT NULL;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `qwiklabs-gcp-02-3205ce60aeba.emergency_response.response_time_model`,
    (
      SELECT
        call_id,
        call_type,
        location,
        weather_condition,
        day_of_week,
        time_of_day,
        traffic_level,
        distance_to_station,
        units_available,
        response_time
      FROM
        `qwiklabs-gcp-02-3205ce60aeba.emergency_response.emergency_calls_response_times`
      WHERE
        response_time IS NOT NULL
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.741761,4.739538,0.014721,1.48122,0.831071,0.831072


In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.PREDICT(MODEL `qwiklabs-gcp-02-3205ce60aeba.emergency_response.response_time_model`,
    (
      WITH synthetic_data AS (
        SELECT
          12345 AS call_id,
          'Medical' AS call_type,
          'Downtown' AS location,
          'Clear' AS weather_condition,
          'Monday' AS day_of_week,
          14 AS time_of_day,
          'Moderate' AS traffic_level,
          3.5 AS distance_to_station,
          2 AS units_available
      )
      SELECT * FROM synthetic_data
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_id,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,811.560615,12345,Medical,Downtown,Clear,Monday,14,Moderate,3.5,2
